### Сравнение трансферного обучения с традиционными моделями в задаче выявления сетевых атак

Трансферное обучение — это метод машинного обучения, при котором модель, обученная на одной задаче, повторно используется для другой, схожей задачи. Вместо того, чтобы обучать модель с нуля, используется уже существующая модель, адаптируя её к новой задаче. Это помогает сократить время обучения и улучшить результаты, особенно если у новой задачи недостаточно данных.

В работе для трансферного обучения используются данные из открытых наборов о кибератаках UNSW-NB15 и Bot-IoT. В работе будет использоваться только 5% данных, набор рекомендованный самими составителями.

Набор UNSW-NB15 содержит сетевые данные, собранные с помощью генератора пакетов, генерирующего нормальный трафик и различные типы атак.

Набор Bot-IoT содержит большое количество реальных данных, собранных в области Интернета вещей с симуляцией нормального и атакующего трафика.

Таким образом, эти наборы имеют различия и при использовании модели, обученной на одном, будет низкая эффективность обнаружения угроз при валидации данных из второго набора. Поэтому из-за схожести задачи  данные можно исользовать при трансферном обучении, которое в данном случае может быть успешным.

Исходные данные обоих файлов сравниваются по признакам, выявляются общие и только на основе них составляются наборы для обучения:

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd

# загрузка файла с признаками
df_features = pd.read_csv('NUSW-NB15_features.csv', encoding='latin1')

# Извлечение заголовков
headers = df_features["Name"].tolist()

# загрузка основных файлов с данными
df1 = pd.read_csv('UNSW-NB15_1.csv', header=None, names=headers)
df2 = pd.read_csv('UNSW-NB15_2.csv', header=None, names=headers)
df3 = pd.read_csv('UNSW-NB15_3.csv', header=None, names=headers)
df4 = pd.read_csv('UNSW-NB15_4.csv', header=None, names=headers)

df_main = pd.concat([df1, df2, df3, df4], ignore_index=True)

# --- Работа с первым набором данных (UNSW-NB15) ---
columns_to_drop = ['srcip', 'dstip', 'sport', 'dsport', 'Stime', 'Ltime']
df_main = df_main.drop(columns=columns_to_drop, errors='ignore')

# Удаление строк с пропусками (кроме attack_cat)
df_main_cleaned = df_main.dropna(subset=[col for col in df_main.columns if col != 'attack_cat'])

# Разделение данных на признаки и целевую переменную
X_unsw = df_main_cleaned.drop(columns=['Label', 'attack_cat'], errors='ignore')
y_unsw = df_main_cleaned['Label']

# --- Работа со вторым набором данных (BoT-IoT) ---
file_paths = [
    'UNSW_2018_IoT_Botnet_Full5pc_1.csv',
    'UNSW_2018_IoT_Botnet_Full5pc_2.csv',
    'UNSW_2018_IoT_Botnet_Full5pc_3.csv',
    'UNSW_2018_IoT_Botnet_Full5pc_4.csv'
]
dataframes = []
for file in file_paths:
    df = pd.read_csv(file)
    dataframes.append(df)

# Объединение файлов
df_bot = pd.concat(dataframes, ignore_index=True)

# Приведение признаков BoT-IoT к признакам UNSW-NB15
unsw_nb15_features = X_unsw.columns.tolist()  # Признаки первого набора
common_features = list(set(df_bot.columns).intersection(unsw_nb15_features))
common_features.append('attack')  # Добавление целевой переменной
df_bot = df_bot[common_features]
df_bot.rename(columns={'attack': 'Label'}, inplace=True)

# Разделение данных на признаки и целевую переменную
X_bot = df_bot.drop(columns=['Label'], errors='ignore')
y_bot = df_bot['Label']

# --- Выравнивание признаков ---
common_features = list(set(X_unsw.columns).intersection(X_bot.columns))
X_unsw = X_unsw[common_features]
X_bot = X_bot[common_features]

# --- Предобработка данных ---
categorical_features = X_unsw.select_dtypes(include=['object']).columns.tolist()
numerical_features = X_unsw.select_dtypes(include=['int64', 'float64']).columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)

pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# Обработка UNSW-NB15
X_unsw_train, X_unsw_test, y_unsw_train, y_unsw_test = train_test_split(X_unsw, y_unsw, test_size=0.2, random_state=42)
X_unsw_train_processed = pipeline.fit_transform(X_unsw_train)
X_unsw_test_processed = pipeline.transform(X_unsw_test)

# Обработка BoT-IoT
X_bot_train, X_bot_test, y_bot_train, y_bot_test = train_test_split(X_bot, y_bot, test_size=0.2, random_state=42)
X_bot_train_processed = pipeline.transform(X_bot_train)
X_bot_test_processed = pipeline.transform(X_bot_test)

# Проверка размеров
print(f"Размер тренировочного набора UNSW-NB15: {X_unsw_train_processed.shape}")
print(f"Размер тестового набора UNSW-NB15: {X_unsw_test_processed.shape}")
print(f"Размер тренировочного набора BoT-IoT: {X_bot_train_processed.shape}")
print(f"Размер тестового набора BoT-IoT: {X_bot_test_processed.shape}")

/var/folders/7m/yq3bdqw12mj3g98dqwm4yk180000gn/T/ipykernel_25606/175988168.py:14: DtypeWarning: Columns (1,3,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('UNSW-NB15_1.csv', header=None, names=headers)
/var/folders/7m/yq3bdqw12mj3g98dqwm4yk180000gn/T/ipykernel_25606/175988168.py:15: DtypeWarning: Columns (3,39,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('UNSW-NB15_2.csv', header=None, names=headers)
/var/folders/7m/yq3bdqw12mj3g98dqwm4yk180000gn/T/ipykernel_25606/175988168.py:41: DtypeWarning: Columns (7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/var/folders/7m/yq3bdqw12mj3g98dqwm4yk180000gn/T/ipykernel_25606/175988168.py:41: DtypeWarning: Columns (7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/var/folders/7m/yq3bdqw12mj3g98dqwm4yk180000gn/T/ipykernel_25606/17598816

Размер тренировочного набора UNSW-NB15: (869762, 153)
Размер тестового набора UNSW-NB15: (217441, 153)
Размер тренировочного набора BoT-IoT: (2934817, 153)
Размер тестового набора BoT-IoT: (733705, 153)


Обучение модели на данных UNSW-NB15:

In [17]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import (
    classification_report,
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    roc_auc_score,
    matthews_corrcoef
)

# Построение базовой нейросети
input_dim = X_unsw_train_processed.shape[1]
model = models.Sequential([
    layers.Dense(128, activation='relu', input_dim=input_dim),
    layers.Dropout(0.3),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')  # Для бинарной классификации
])

# Компиляция модели
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Обучение
history = model.fit(X_unsw_train_processed, y_unsw_train, epochs=10, batch_size=32, validation_split=0.2)

# Оценка на тестовых данных
y_unsw_pred_probs = model.predict(X_unsw_test_processed)  # Предсказания вероятностей
y_unsw_pred_nn = (y_unsw_pred_probs > 0.5).astype("int32")  # Бинарные предсказания

# Метрики
accuracy_1 = accuracy_score(y_unsw_test, y_unsw_pred_nn)
f1_1 = f1_score(y_unsw_test, y_unsw_pred_nn)
precision_1 = precision_score(y_unsw_test, y_unsw_pred_nn)
recall_1 = recall_score(y_unsw_test, y_unsw_pred_nn)
roc_auc_1 = roc_auc_score(y_unsw_test, y_unsw_pred_probs)
mcc_1 = matthews_corrcoef(y_unsw_test, y_unsw_pred_nn)
cr_1 = classification_report(y_unsw_test, y_unsw_pred_nn, output_dict=True)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
21745/21745 ━━━━━━━━━━━━━━━━━━━━ 29s 1ms/step - accuracy: 0.9819 - loss: 0.0673 - val_accuracy: 0.9842 - val_loss: 0.0587
Epoch 2/10
21745/21745 ━━━━━━━━━━━━━━━━━━━━ 32s 1ms/step - accuracy: 0.9833 - loss: 0.0612 - val_accuracy: 0.9842 - val_loss: 0.0578
Epoch 3/10
21745/21745 ━━━━━━━━━━━━━━━━━━━━ 33s 1ms/step - accuracy: 0.9832 - loss: 0.0590 - val_accuracy: 0.9823 - val_loss: 0.0559
Epoch 4/10
21745/21745 ━━━━━━━━━━━━━━━━━━━━ 29s 1ms/step - accuracy: 0.9830 - loss: 0.0584 - val_accuracy: 0.9842 - val_loss: 0.0533
Epoch 5/10
21745/21745 ━━━━━━━━━━━━━━━━━━━━ 29s 1ms/step - accuracy: 0.9834 - loss: 0.0565 - val_accuracy: 0.9842 - val_loss: 0.0541
Epoch 6/10
21745/21745 ━━━━━━━━━━━━━━━━━━━━ 33s 2ms/step - accuracy: 0.9836 - loss: 0.0544 - val_accuracy: 0.9825 - val_loss: 0.0555
Epoch 7/10
21745/21745 ━━━━━━━━━━━━━━━━━━━━ 29s 1ms/step - accuracy: 0.9834 - loss: 0.0542 - val_accuracy: 0.9842 - val_loss: 0.0499
Epoch 8/10
21745/21745 ━━━━━━━━━━━━━━━━━━━━ 27s 1ms/step - accuracy: 

Сохранение изначальной модели:

In [18]:
# Сохранение модели
model.save("baseline_model.h5")

Применение трансферного обучения на BoT-IoT к созданной модели:

In [19]:
# --- Трансферное обучение на BoT-IoT ---
# Заморозка всех слоев, кроме последних
for layer in model.layers[:-1]:
    layer.trainable = False  # Замораживаем слои для сохранения обученных весов

# Добавление новых слоев, если необходимо адаптировать модель
# В данном случае мы используем ту же архитектуру, так что новые слои не добавляются

# Компиляция модели для дообучения
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Дообучение на BoT-IoT
history_transfer = model.fit(X_bot_train_processed, y_bot_train, epochs=5, batch_size=32, validation_split=0.2)

# Оценка модели на тестовом наборе BoT-IoT
y_bot_pred_probs = model.predict(X_bot_test_processed)  # Предсказания вероятностей
y_bot_pred_nn = (y_bot_pred_probs > 0.5).astype("int32")  # Бинарные предсказания

# Метрики для BoT-IoT
accuracy_2 = accuracy_score(y_bot_test, y_bot_pred_nn)
f1_2 = f1_score(y_bot_test, y_bot_pred_nn)
precision_2 = precision_score(y_bot_test, y_bot_pred_nn)
recall_2 = recall_score(y_bot_test, y_bot_pred_nn)
roc_auc_2 = roc_auc_score(y_bot_test, y_bot_pred_probs)
mcc_2 = matthews_corrcoef(y_bot_test, y_bot_pred_nn)
cr_2 = classification_report(y_bot_test, y_bot_pred_nn, output_dict=True)



Epoch 1/5
73371/73371 ━━━━━━━━━━━━━━━━━━━━ 162s 2ms/step - accuracy: 0.9836 - loss: 0.2106 - val_accuracy: 0.9999 - val_loss: 0.0044
Epoch 2/5
73371/73371 ━━━━━━━━━━━━━━━━━━━━ 176s 2ms/step - accuracy: 0.9999 - loss: 0.0032 - val_accuracy: 0.9999 - val_loss: 0.0035
Epoch 3/5
73371/73371 ━━━━━━━━━━━━━━━━━━━━ 157s 2ms/step - accuracy: 0.9999 - loss: 0.0025 - val_accuracy: 0.9999 - val_loss: 0.0041
Epoch 4/5
73371/73371 ━━━━━━━━━━━━━━━━━━━━ 153s 2ms/step - accuracy: 0.9999 - loss: 0.0026 - val_accuracy: 0.9999 - val_loss: 0.0037
Epoch 5/5
73371/73371 ━━━━━━━━━━━━━━━━━━━━ 158s 2ms/step - accuracy: 0.9999 - loss: 0.0023 - val_accuracy: 0.9999 - val_loss: 0.0037
22929/22929 ━━━━━━━━━━━━━━━━━━━━ 26s 1ms/step


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

Сохранение дообученной модели:

In [20]:
# Сохранение модели
model.save("transfer_model.h5")

Метрики двух обучений одной модели сохраняются в один файл:

In [23]:
# Метрики для первого обучения (UNSW-NB15)
metrics_unsw = {
    "Metric": ["Accuracy", "F1 Score", "Precision", "Recall", "ROC-AUC", "MCC"],
    "UNSW-NB15": [accuracy_1, f1_1, precision_1, recall_1, roc_auc_1, mcc_1]
}

# Метрики для второго обучения (BoT-IoT)
metrics_bot = {
    "Metric": ["Accuracy", "F1 Score", "Precision", "Recall", "ROC-AUC", "MCC"],
    "BoT-IoT": [accuracy_2, f1_2, precision_2, recall_2, roc_auc_2, mcc_2]
}

# Создание датафрейма
metrics_df = pd.DataFrame({
    "Metric": metrics_unsw["Metric"],
    "UNSW-NB15": metrics_unsw["UNSW-NB15"],
    "BoT-IoT": metrics_bot["BoT-IoT"]
})

# Сохранение в CSV
metrics_df.to_csv("model_metrics_comparison.csv", index=False)

Сохранение отчетов модели в файлы:

In [24]:
# Преобразование в DataFrame
report_df = pd.DataFrame(cr_1).transpose()
# Сохранение в файл
report_df.to_csv("classification_report_1.csv", index=True)

report_df = pd.DataFrame(cr_2).transpose()
# Сохранение в файл
report_df.to_csv("classification_report_2.csv", index=True)


Обучение классических методов машинного обучения на лимитированных данных:

In [25]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import (
    classification_report,
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    roc_auc_score,
    matthews_corrcoef
)
import pandas as pd

# Классические методы машинного обучения
classifiers = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "Support Vector Machine": SVC(probability=True, random_state=42)
}

# Словарь для сохранения метрик
metrics_comparison = []

# Обучение и оценка моделей
for name, clf in classifiers.items():
    print(f"Обучение модели: {name}")
    
    # Обучение модели
    clf.fit(X_bot_train_processed, y_bot_train)
    
    # Предсказания
    y_pred = clf.predict(X_bot_test_processed)
    y_pred_probs = (
        clf.predict_proba(X_bot_test_processed)[:, 1] 
        if hasattr(clf, "predict_proba") else None
    )
    
    # Метрики
    accuracy = accuracy_score(y_bot_test, y_pred)
    f1 = f1_score(y_bot_test, y_pred)
    precision = precision_score(y_bot_test, y_pred)
    recall = recall_score(y_bot_test, y_pred)
    roc_auc = roc_auc_score(y_bot_test, y_pred_probs) if y_pred_probs is not None else None
    mcc = matthews_corrcoef(y_bot_test, y_pred)
    
    # Сохранение метрик
    metrics_comparison.append({
        "Model": name,
        "Accuracy": accuracy,
        "F1 Score": f1,
        "Precision": precision,
        "Recall": recall,
        "ROC-AUC": roc_auc if roc_auc is not None else "N/A",
        "MCC": mcc
    })
    
    # Вывод classification_report
    print(f"\nClassification Report для модели {name}:\n")
    print(classification_report(y_bot_test, y_pred))
    
    # Сохранение classification_report в файл
    report = classification_report(y_bot_test, y_pred, output_dict=True)
    report_df = pd.DataFrame(report).transpose()
    report_df.to_csv(f"classification_report_lim_data_{name.replace(' ', '_')}.csv", index=True)

# Создание датафрейма с метриками
metrics_df = pd.DataFrame(metrics_comparison)

# Сохранение метрик в CSV
metrics_csv_path = "model_metrics_classical_lim_data_comparison.csv"
metrics_df.to_csv(metrics_csv_path, index=False)

Обучение модели: Logistic Regression

Classification Report для модели Logistic Regression:



/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       101
           1       1.00      1.00      1.00    733604

    accuracy                           1.00    733705
   macro avg       0.50      0.50      0.50    733705
weighted avg       1.00      1.00      1.00    733705



/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

Обучение модели: Random Forest

Classification Report для модели Random Forest:

              precision    recall  f1-score   support

           0       0.92      0.93      0.93       101
           1       1.00      1.00      1.00    733604

    accuracy                           1.00    733705
   macro avg       0.96      0.97      0.96    733705
weighted avg       1.00      1.00      1.00    733705

Обучение модели: Support Vector Machine

Classification Report для модели Support Vector Machine:

              precision    recall  f1-score   support

           0       0.96      0.54      0.70       101
           1       1.00      1.00      1.00    733604

    accuracy                           1.00    733705
   macro avg       0.98      0.77      0.85    733705
weighted avg       1.00      1.00      1.00    733705



Соединение всех полученных метрик в один файл:

In [28]:
import pandas as pd

# Пути к двум CSV-файлам
file1 = "model_metrics_comparison.csv"  # Файл с метриками UNSW-NB15 и BoT-IoT
file2 = "model_metrics_classical_lim_data_comparison.csv"  # Файл с метриками классических моделей

# Загрузка данных из файлов
df1 = pd.read_csv(file1)  # Метрики для UNSW-NB15 и BoT-IoT
df2 = pd.read_csv(file2)  # Метрики для классических методов машинного обучения

# Преобразование df1: разделяем данные для UNSW-NB15 и BoT-IoT как отдельные модели
df1_unsw = df1[["Metric", "UNSW-NB15"]].rename(columns={"UNSW-NB15": "Value"})
df1_unsw["Model"] = "UNSW-NB15"

df1_bot = df1[["Metric", "BoT-IoT"]].rename(columns={"BoT-IoT": "Value"})
df1_bot["Model"] = "BoT-IoT"

# Объединяем метрики из первого файла
df1_combined = pd.concat([df1_unsw, df1_bot], ignore_index=True)

# Преобразование второго файла (модели уже идут строками)
df2_melted = df2.melt(id_vars=["Model"], var_name="Metric", value_name="Value")

# Объединяем данные из обоих файлов
combined = pd.concat([df1_combined, df2_melted], ignore_index=True)

# Преобразуем таблицу в финальный формат: строки — модели, столбцы — метрики
final_df = combined.pivot(index="Model", columns="Metric", values="Value").reset_index()

# Упорядочиваем столбцы
final_df = final_df[["Model", "Accuracy", "F1 Score", "Precision", "Recall", "ROC-AUC", "MCC"]]

# Вывод объединенной таблицы в консоль
print("\nОбъединенные метрики моделей:\n")
print(final_df)

# Сохранение таблицы в файл
output_file = "final_metrics_table.csv"
final_df.to_csv(output_file, index=False)


Объединенные метрики моделей:

Metric                   Model  Accuracy  F1 Score  Precision    Recall  \
0                      BoT-IoT  0.999862  0.999931   0.999862  1.000000   
1          Logistic Regression  0.999862  0.999931   0.999862  1.000000   
2                Random Forest  0.999980  0.999990   0.999990  0.999989   
3       Support Vector Machine  0.999935  0.999967   0.999937  0.999997   
4                    UNSW-NB15  0.984313  0.603510   0.630556  0.578689   

Metric   ROC-AUC       MCC  
0       0.862010  0.000000  
1       0.836210  0.000000  
2       0.999979  0.926109  
3       0.941980  0.724852  
4       0.956345  0.596095  


Трансферное обучение в данном случае не доказало свою эффективность. Модель случайного леса имеет наилучшие результаты.

In [29]:
cr1 = pd.read_csv('classification_report_2.csv')
cr2 = pd.read_csv('classification_report_lim_data_Logistic_Regression.csv')
cr3 = pd.read_csv('classification_report_lim_data_Random_Forest.csv')
cr4 = pd.read_csv('classification_report_lim_data_Support_Vector_Machine.csv')
print("Transfer learning:")
print(cr1)
print("Logistic Regression:")
print(cr2)
print("Random_Forest:")
print(cr3)
print("Support_Vector_Machine:")
print(cr4)


Transfer learning:
     Unnamed: 0  precision    recall  f1-score        support
0             0   0.000000  0.000000  0.000000     101.000000
1             1   0.999862  1.000000  0.999931  733604.000000
2      accuracy   0.999862  0.999862  0.999862       0.999862
3     macro avg   0.499931  0.500000  0.499966  733705.000000
4  weighted avg   0.999725  0.999862  0.999794  733705.000000
Logistic Regression:
     Unnamed: 0  precision    recall  f1-score        support
0             0   0.000000  0.000000  0.000000     101.000000
1             1   0.999862  1.000000  0.999931  733604.000000
2      accuracy   0.999862  0.999862  0.999862       0.999862
3     macro avg   0.499931  0.500000  0.499966  733705.000000
4  weighted avg   0.999725  0.999862  0.999794  733705.000000
Random_Forest:
     Unnamed: 0  precision    recall  f1-score       support
0             0   0.921569  0.930693  0.926108     101.00000
1             1   0.999990  0.999989  0.999990  733604.00000
2      accuracy   

Наилучшая по всем показателям модель случайного леса.